<a href="https://colab.research.google.com/github/nacho-gonz/NLP-Entregas-2024/blob/main/TP1_2024/TP_NLP_TUIA_2024_NOIR_GONZALEZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [1]:
!pip install -U deep-translator
!pip install sentence-transformers
!pip install torch
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00


In [16]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import torch
from deep_translator import GoogleTranslator
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Text, Button, Output, VBox, IntSlider, HBox
from gliner import GLiNER
import re
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from tqdm import TqdmExperimentalWarning
warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)
from torch import Tensor
from pandas import DataFrame

# Entrenamiento y carga de modelos

In [3]:
modelo_ner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
modelo_ner.eval()

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(250105, 768, padding_idx=0)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-11): 12 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=768, out_features=768, bias=True)
                    (key_proj): Linear(in_features=768, out_features=768, bias=True)
                    (value_proj): Linear(in_features=768, out_features=768, bias=True)
                    (pos_dropout): StableDropout()
                    (dropout): StableDropout()
                  )
                  (output): Debert

In [4]:
modelo_semantic_multil = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Etiquetas de la clasificación
labels = [(0, "sad"),(1, "neutral"), (2, "happy")]

dataset_sent = pd.read_csv('https://github.com/nacho-gonz/NLP-Practica-archivos-2024/raw/main/TP1_2024/dataset_sent.csv')

# Es necesario pasar el tipo de dato a lista para el train_test_split
X = dataset_sent['sentence'].to_list()
y = dataset_sent['label'].to_list()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hacemos el embedding de todas las frases del dataset.
X_train_vectorized = modelo_semantic_multil.encode(X_train)
X_test_vectorized = modelo_semantic_multil.encode(X_test)

In [6]:
modelo_LR = LogisticRegression(max_iter=3000, solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística: 0.9259259259259259
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       0.84      0.94      0.89        17
           2       0.94      0.85      0.89        20

    accuracy                           0.93        54
   macro avg       0.93      0.93      0.93        54
weighted avg       0.93      0.93      0.93        54



# Análisis de sentimiento y recomendación de peliculas, juegos de mesa y libros

In [7]:
peliculas = pd.read_csv('https://github.com/nacho-gonz/NLP-Practica-archivos-2024/raw/main/TP1_2024/peliculas.csv')
juegos_de_mesa = pd.read_csv('https://github.com/nacho-gonz/NLP-Practica-archivos-2024/raw/main/TP1_2024/juegos_de_mesa.csv', sep=',')
libros = pd.read_csv('https://github.com/nacho-gonz/NLP-Practica-archivos-2024/raw/main/TP1_2024/libros.csv',sep=',')

In [14]:
def sentiment_pipeline(text: str) -> str:
  """
  Clasifica el sentimiento de una frase en triste, neutral y feliz.
  """

  # Pasamos todo el texto a lowercase para no tener problemas con mayusculas, lo vectorizamos o le hacemos el embedding y predecimos la clase
  new_phrases_lower = [text.lower()]
  new_phrases_vectorized = modelo_semantic_multil.encode(new_phrases_lower)
  new_predictions = modelo_LR.predict(new_phrases_vectorized)

  # Cambiamos la etiqueta para que no sean números
  if new_predictions[0] == 0:
    sent_clasif = 'Triste'
  elif new_predictions[0] == 1:
    sent_clasif = 'Neutral'
  else:
    sent_clasif = 'Feliz'

  return sent_clasif



In [10]:
def string_to_array(tensor_str: str) -> Tensor:
  """
  Cambia el tipo de dato de un string a un tensor de PyTorch.
  """
  tensor_str = re.sub(r"^tensor\(|\)$", "", tensor_str)
  tensor_str = tensor_str.replace("[", "").replace("]", "")
  # Eliminar saltos de línea y espacios adicionales
  tensor_str = tensor_str.replace("\n", "").replace(" ", "")
  # Convertir el string en una lista de floats
  tensor_list = [float(x) for x in tensor_str.split(",")]
  # Convertir a tensor de PyTorch o a array de NumPy
  return torch.tensor(tensor_list)

# Utilizamos la función string_to_array para poder utilizar los embeddings.
peliculas['embedding'] = peliculas['embedding'].apply(string_to_array)
juegos_de_mesa['embedding'] = juegos_de_mesa['embedding'].apply(string_to_array)
libros['embedding'] = libros['embedding'].apply(string_to_array)
libros['embedding_author'] = libros['embedding_author'].apply(string_to_array)
peliculas['embedding_actor_director'] = peliculas['embedding_actor_director'].apply(string_to_array)

In [11]:
# Cambiamos el nombre de las columnas de los datasets para un manejo más cómodo.
peliculas.rename(columns={'Title': 'titulo'}, inplace=True)
juegos_de_mesa.rename(columns={'game_name': 'titulo'}, inplace=True)
libros.rename(columns={'Title': 'titulo'}, inplace=True)

# Pasamos los embeddings como una lista de respuestas para la búsqueda semántica.
respuestas_juegos_mesa = juegos_de_mesa['embedding'].to_list()
respuestas_libros = libros['embedding'].to_list()
respuestas_peliculas = peliculas['embedding'].to_list()
respuestas_libros_autores = libros['embedding_author'].to_list()
respuestas_peliculas_directores = peliculas['embedding_actor_director'].to_list()

In [17]:
# Se definen los widgets de entrada para capturar el sentimiento y la preferencia de la persona
sent_pers = Text(description="Sentimiento:",placeholder='Cómo se siente?')
preferencia_pers = Text(description="Preferencia:", placeholder='Que desea ver/leer/jugar?')
boton_confirmar = Button(description="Confirmar")
output = Output()
display_box = VBox()

# Se inicializa una variable para almacenar el estado de las preferencias
preferencias_confirmadas = False

# Función que se usa para mostrar las preferencias confirmadas y establecer el estado
def confirmar_preferencias(b):
    global preferencias_confirmadas
    with output:
        output.clear_output()
        print(f"Sentimiento: {sentiment_pipeline(sent_pers.value)}")
        print(f"Preferencia: {preferencia_pers.value}")
    preferencias_confirmadas = True
    seleccion_automatica()

# Se asigna la función de confirmación al botón de confirmar
boton_confirmar.on_click(confirmar_preferencias)


# Se definen los widgets de selección y botones de categorías
boton_peliculas = Button(description="Películas")
boton_juegos = Button(description="Juegos de Mesa")
boton_libros = Button(description="Libros")
boton_todos = Button(description="Todos")

slider_peliculas = IntSlider(min=0, max=5, step=1, description="Cantidad:")
slider_juegos = IntSlider(min=0, max=5, step=1, description="Cantidad:")
slider_libros = IntSlider(min=0, max=5, step=1, description="Cantidad:")
slider_todos = IntSlider(min=0, max=5, step=1, description="Cantidad:")

# Se implementa una función para encontrar entidades específicas en el texto proporcionado
def encontrar_entidades(texto):
    labels_ner = ['actor', 'director', 'author']

    entities = modelo_ner.predict_entities(texto, labels_ner, threshold=0.5)

    return entities

# Función que se utiliza para traducir el texto de entrada y generar embeddings solo después de la confirmación
def obtener_embedding() -> tuple[Tensor, str] | None:
    """
    Función que obtiene el embedding de la preferencia del usuario.
    """

    if preferencias_confirmadas:
        # Se traduce la preferencia a inglés para realizar la búsqueda
        pref_translated = GoogleTranslator(source='auto', target='en').translate(preferencia_pers.value)

        entidades = encontrar_entidades(pref_translated)
        # Se crean los embeddings de la preferencia combinados

        query_embedding = modelo_semantic_multil.encode(pref_translated, convert_to_tensor=True)
        return query_embedding, entidades
    else:
        return None

def limpiar_interfaz() -> None:
    """
    Función que limpia la interfaz de usuario.
    """

    display_box.children = []
    output.clear_output()

# Función que se utiliza para mostrar las recomendaciones de cada categoría
def mostrar_categoria(categoria_df: DataFrame, slider:IntSlider, resultados:list, tipo:str) -> None:
    """
    Limpia la interfaz, crea un slider y muestra la cantidad de resultados seleccionados dependiendo de el número seleccionado en el slider.
    """

    limpiar_interfaz()
    display_box.children = [HBox([slider, output])]

    def actualizar_resultados(change):
        slider.unobserve(actualizar_resultados, names='value')
        cantidad = slider.value
        indices = [hit['corpus_id'] for hit in resultados[:cantidad]]
        with output:
            output.clear_output()
            for idx in indices:
                print(f"{tipo} {categoria_df.iloc[idx]['titulo']}")
        slider.observe(actualizar_resultados, names='value')

    slider.observe(actualizar_resultados, names='value')


# Funciones de categoría que muestran los resultados filtrados por categoría al hacer clic en un botón
def mostrar_peliculas(query_embedding_func=None, respuestas_peliculas_func=None) -> None:
    """
    Función que muestra las películas filtradas por el embedding de la preferencia del usuario, si el usuario ingresa un actor o director.
    """
    # Si existe el embedding de la preferencia del usuario se hace la búsqueda semántica y se llama a mostrar categoria con el dataset, el slider, el resultado de la búsqueda y la primer parte del print.
    if query_embedding_func is not None:
        resultados_peliculas_func = util.semantic_search(query_embedding_func, respuestas_peliculas_func, top_k=5)[0]
        categorias = [peliculas.iloc[hit['corpus_id']]['titulo'] for hit in resultados_peliculas_func]
        mostrar_categoria(peliculas, slider_peliculas, resultados_peliculas_func, 'Pelicula:')

def mostrar_peliculas_general(b:None) -> None:
    """
    Función que muestra las películas filtradas por el embedding de la preferencia del usuario, si el usuario presiona el boton películas.
    """
    query_embedding, _ = obtener_embedding()
    # Si existe el embedding de la preferencia del usuario se hace la búsqueda semántica y se llama a mostrar categoria con el dataset, el slider, el resultado de la búsqueda y la primer parte del print.
    if query_embedding is not None:
        resultados_peliculas = util.semantic_search(query_embedding, respuestas_peliculas, top_k=5)[0]
        categorias = [peliculas.iloc[hit['corpus_id']]['titulo'] for hit in resultados_peliculas]
        mostrar_categoria(peliculas, slider_peliculas, resultados_peliculas, 'Pelicula:')

def mostrar_juegos(b: None) -> None:
    """
    Función que muestra los juegos de mesa filtrados por el embedding de la preferencia del usuario, si el usuario presiona el boton juego de mesa.
    """
    query_embedding, _ = obtener_embedding()
    # Si existe el embedding de la preferencia del usuario se hace la búsqueda semántica y se llama a mostrar categoria con el dataset, el slider, el resultado de la búsqueda y la primer parte del print.
    if query_embedding is not None:
        resultados_juegos = util.semantic_search(query_embedding, respuestas_juegos_mesa, top_k=5)[0]
        categorias = [juegos_de_mesa.iloc[hit['corpus_id']]['titulo'] for hit in resultados_juegos]
        mostrar_categoria(juegos_de_mesa, slider_juegos, resultados_juegos, 'Juegos de mesa:')


def mostrar_libros(query_embedding_func=None, respuestas_libros_func=None) -> None:
    """
    Función que muestra los libros filtrados por el embedding de la preferencia del usuario, si el usuario ingresa el autor de un libro.
    """
    # Si existe el embedding de la preferencia del usuario se hace la búsqueda semántica y se llama a mostrar categoria con el dataset, el slider, el resultado de la búsqueda y la primer parte del print.
    if query_embedding_func is not None:
        resultados_libros = util.semantic_search(query_embedding_func, respuestas_libros_func, top_k=5)[0]
        categorias = [libros.iloc[hit['corpus_id']]['titulo'] for hit in resultados_libros]
        mostrar_categoria(libros, slider_libros, resultados_libros, 'Libro:')

def mostrar_libros_general(b:None) -> None:
    """
    Función que muestra los libros filtrados por el embedding de la preferencia del usuario, si el usuario presiona el boton libros.
    """
    query_embedding, _ = obtener_embedding()
    # Si existe el embedding de la preferencia del usuario se hace la búsqueda semántica y se llama a mostrar categoria con el dataset, el slider, el resultado de la búsqueda y la primer parte del print.
    if query_embedding is not None:
        resultados_libros = util.semantic_search(query_embedding, respuestas_libros, top_k=5)[0]
        categorias = [libros.iloc[hit['corpus_id']]['titulo'] for hit in resultados_libros]
        mostrar_categoria(libros, slider_libros, resultados_libros, 'Libro:')

# Función que muestra todas las categorías
def mostrar_todos(b:None) -> None:
    """
    Función que muestra todas las categorías filtradas por el embedding de la preferencia del usuario, si el usuario presiona el boton todos.
    """
    query_embedding, _ = obtener_embedding()
    # Si el embedding existe, limpia la interfaz, crea un slider y llama a actualizar todos, para que una vez se interactue con el slider devuelva la cantidad seleccionada en el mismo de peliculas, juegos de mesa y libros.
    if query_embedding is not None:
        limpiar_interfaz()
        display_box.children = [HBox([slider_todos, output])]

        def actualizar_todos(change:None) -> None:
            """
            Función que actualiza la cantidad de resultados seleccionados en el slider.
            """

            # Se hace la búsqueda semántica dependiendo el valor seleccionado en el slider y se guarda en una lista los nombres de cada uno de los resultados para asi mostrarlos de una manera más compacta.
            slider_todos.unobserve(actualizar_todos, names='value')
            cantidad = slider_todos.value
            resultados_peliculas = util.semantic_search(query_embedding, respuestas_peliculas, top_k=cantidad)[0]
            resultados_juegos = util.semantic_search(query_embedding, respuestas_juegos_mesa, top_k=cantidad)[0]
            resultados_libros = util.semantic_search(query_embedding, respuestas_libros, top_k=cantidad)[0]
            with output:
                output.clear_output()

                for i in range(cantidad):
                    elementos = []

                    if i < len(resultados_peliculas):
                        titulo_pelicula = peliculas.iloc[resultados_peliculas[i]['corpus_id']]['titulo']
                        elementos.append(f"Pelicula: {titulo_pelicula}")

                    if i < len(resultados_juegos):
                        titulo_juego = juegos_de_mesa.iloc[resultados_juegos[i]['corpus_id']]['titulo']
                        elementos.append(f"Juego de Mesa: {titulo_juego}")

                    if i < len(resultados_libros):
                        titulo_libro = libros.iloc[resultados_libros[i]['corpus_id']]['titulo']
                        elementos.append(f"Libro: {titulo_libro}")

                    for elemento in elementos:
                        print(elemento)
                    print("---")

            slider_todos.observe(actualizar_todos, names='value')

        slider_todos.observe(actualizar_todos, names='value')
        actualizar_todos(None)

# Se conecta cada botón de categoría a sus funciones de visualización
def seleccion_automatica() -> None:
    """
    Función que dependiendo las entidades encontradas en la preferencia restringe la búsqueda semántica.
    """
    query_embedding, entidades = obtener_embedding()

    # Si existe el embedding y no se encuentran entidades entonces se muestran los botones, en caso de encontrar una entidad se muestra solamente la respuesta en base a si es autor, director o actor.
    if query_embedding is not None:
        if not entidades:
            display_box.children = [
                HBox([boton_peliculas, boton_juegos, boton_libros, boton_todos]), output
            ]
        else:
            encontrados = [entidad['label'] for entidad in entidades]
            if 'actor' in encontrados or 'director' in encontrados:
                mostrar_peliculas(query_embedding, respuestas_peliculas_directores)
            elif 'author' in encontrados:
                mostrar_libros(query_embedding, respuestas_libros_autores)

# Botones para seleccionar que se recomienda.

boton_peliculas.on_click(mostrar_peliculas_general)
boton_juegos.on_click(mostrar_juegos)
boton_libros.on_click(mostrar_libros_general)
boton_todos.on_click(mostrar_todos)

# Se estructura la interfaz de usuario principal con todos los elementos configurados
ui = VBox([
          HBox([sent_pers, preferencia_pers, boton_confirmar]),
          display_box, output
      ])

display(ui)
